# 🎨 ai-gen - LoRA Training & Image Generation

**Train LoRAs and generate ultra-realistic images with NO restrictions**

---

## 🚀 Quick Start

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Run Cell 1**: Setup (installs everything)
3. **Access GUI**: Click the ngrok URL when it appears

---

In [ ]:
# CELL 1: Complete Setup & Launch
# Run this cell - takes about 10-15 minutes

print("🚀 Starting MASUKA setup...\n")

# Clean up any previous installations to force fresh download
print("🧹 Cleaning previous installations...")
!rm -rf /content/CharForgex /content/ai-gen 2>/dev/null || true

# Clone the latest version from GitHub
print("📥 Downloading latest version from GitHub...")
!git clone https://github.com/SamuelD27/ai-gen.git /content/ai-gen

# Change to repo directory
import os
os.chdir('/content/ai-gen')

# Always pull latest changes to ensure we have the newest code
print("🔄 Ensuring latest version...")
!git pull origin main

# Download and run the setup script
print("⚙️  Running setup script...\n")
!python masuka_colab_setup.py